# InstaScams

This notebook is dedicated to combatting scam accounts.
The inspiration is taken from `passports.saime.vzl`, a site
with 70k followers which pretends to be the official [SAIME](http://www.saime.gob.ve/)
website for Venezuelan citizens to get passports and immigration documents.

According to some comments which were deleted, the account is managed by
some people who do work at SAIME, but dozens of different users reported that the
site was fraudulent. The account gains trust by posting valid information about 
the visa process. Then it offers individual chats with more information.
But soon they will suggest that a fee of `$300` to `$3000`
will allow them to process the applications online. If paid, the victim will
never get their documents nor money back.

This site has been reported hundreds of times to Instagram, but no action has yet been taken.


## Indicators of Fraudulent Sites
Some things that might indicate that a site is fraudulent

* The account's name has been changed many times
* User comments are blocked
* Account claims to be "Official" but does is not [verified](https://help.instagram.com/733907830039577)

![saime_account.png](images/saime_account.png)

If you check the '...' "About" you see the account has been renamed many times.
![former_usernames.png](images/former_usernames.png)

# The Basics



In [128]:
import os
import requests
import json
from functools import reduce
from datetime import datetime

## Instaloader
Whenever possible, use instaloader for calls github.com/instaloader/instaloader  
https://instaloader.github.io/as-module.html


In [5]:
!pip install instaloader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 KB 776.1 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for instaloader: filename=instaloader-4.9.4-py3-none-any.whl size=62307 sha256=c88a21bf3a807cb13b82e0cc2fa1a9d46e8a1b37c9bbb5d7028cecbe9723b314
  Stored in directory: /tmp/pip-ephem-wheel-cache-uvkz5qg1/wheels/22/2e/69/a82ee5314382488c557ae65b2702df34821a296524ec897304
Successfully built instaloader


In [6]:
import instaloader

On the commandline, login via `instaloader login` to save creds to a file.
then the following call to load_session_from_file will work.

In [10]:
target_user = 'saime.pasaporte.ve'

L = instaloader.Instaloader()
# user to make posts with
post_user = 'defeatscammers'
# user to read profiles/posts
crawl_profile_user = 'defeatscammers'
# Optionally, login or load session
#L.login(user, PASSWORD)        # (login)
#L.interactive_login(user)      # (ask password on terminal)

L.load_session_from_file(crawl_profile_user)

profile = instaloader.Profile.from_username(L.context, target_user)

posts = profile.get_posts()


Loaded session from /home/jovyan/.config/instaloader/session-sergeidvorak86.


In [15]:
posts.count

163

# Investigations

## Check for comments
This account used to have open comments, but now almost all posts are closed to
comments and have had all comments removed

In [26]:
all_comments = {}
for index, post in enumerate(posts):
    comments = post.get_comments()
    comments_count = post.comments
    print('comments_count', comments_count)
    all_comments[post.shortcode] = comments
    print(comments)
    dl_post = L.download_post(post, target=target_user)
    print(dl_post)
    # print the first, then exit
    if index > 1:
        break

['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']
comments_count 0
[]
saime.pasaporte.ve/2022-06-12_01-56-38_UTC_1.jpg saime.pasaporte.ve/2022-06-12_01-56-38_UTC_2.jpg saime.pasaporte.ve/2022-06-12_01-56-38_UTC_3.jpg [Pasaportes y prórrogas dispon…] json 
True
['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', 

In [36]:
from glob import glob

def load_posts(target_user):
    posts = set(filter(lambda s: isinstance(s, instaloader.Post),
                           (instaloader.load_structure_from_file(L.context, file)
                            for file in (
                                glob(f'{target_user}/*.json.xz') + 
                                glob(f'{target_user}/*.json')))))
    return posts

posts = load_posts(target_user)
print(len(posts))

16


### Check for indicators of scam accounts

commenting_disabled_for_viewer
comments_disabled
is_verified

"transparency_product":"STATE_CONTROLLED_MEDIA"

### Inspect tagged posts

In [53]:
tagged_posts = list(profile.get_tagged_posts())
for tagged_post in tagged_posts:
    print(tagged_post)

<Post CigVEk0OFdm>
<Post CicLyHCD67w>
<Post CiauZWyJqFs>
<Post CiF6ixKueSX>
<Post Chw-2cZDh5q>
<Post ChmGybUunXx>
<Post Cg2fU6CuN09>
<Post Cf9E2JGOuin>
<Post CeZS8qiloJA>
<Post Cc50-pTLote>
<Post CNQ1ek3qEr->
<Post CMXLnVin2j4>


### Likes and Comments
For scam accounts, you will see a large number of likes and comments
being deleted. The victim may delete their like. The scammer will delete
critical comments.

In [58]:
def get_likes_and_comments(posts,
                           max_posts=2,
                           max_likes=2,
                           max_comments=2):
    interactions = {}
    posts_counter = 0
    for post in posts:
        shortcode = post.shortcode
        print(shortcode)
        interactions[shortcode] = {}
        likes_count = post.likes
        comments_count = post.comments
        print(f'post {shortcode} has {likes_count} likes and {comments_count} comments')
        if comments_count:
            comments = list(post.get_comments())
            interactions[shortcode]['comments'] = comments
            comment_counter = 0
            for comment in comments:
                print(comment)
                comment_counter += 1
                if max_comments and (comment_counter > max_comments):
                    break
        if likes_count:
            likes_counter = 0
            # Get profiles of likees
            likes = list(post.get_likes())
            interactions[shortcode]['likes'] = likes
            for like in likes:
                likes_counter += 1
                print('like', like)
                if max_likes and (likes_counter > max_likes):
                    break
        posts_counter += 1
        if max_posts and (posts_counter > max_posts):
            break
    return interactions

In [57]:
tagged_interactions = get_likes_and_comments(tagged_posts)
tagged_interactions[list(tagged_interactions.keys())[0]]

CigVEk0OFdm
post CigVEk0OFdm has 16 likes and 0 comments
like <Profile celsosotelo4 (48196323115)>
like <Profile guzman_roman_ (3975132953)>
like <Profile lourdes.leonett (12992011763)>
CicLyHCD67w
post CicLyHCD67w has 62 likes and 19 comments
PostComment(id=17957171915005113, created_at_utc=datetime.datetime(2022, 9, 13, 9, 59, 37), text='Gracias a ti ya tengo mi cita!!!', owner=<Profile militzaortizjewellery (52570307)>, likes_count=2, answers=<generator object Post.get_comments.<locals>._postcommentanswers at 0x7fb9e0cb2ba0>)
PostComment(id=17981575828628416, created_at_utc=datetime.datetime(2022, 9, 14, 21, 49, 21), text='Que emoción ❤️❤️ que te llegue rápido 🙏🏻🥳gracias por la info la voy a compartir yo ando en las mismas Me toca hacer un vídeo así 😵\u200d💫😁 me pidieron la info full ayer 😖', owner=<Profile biglatinflavours (1081773287)>, likes_count=1, answers=<generator object Post.get_comments.<locals>._postcommentanswers at 0x7fb9e0cb20b0>)
PostComment(id=18215875063087087, crea

{'likes': [<Profile celsosotelo4 (48196323115)>,
  <Profile guzman_roman_ (3975132953)>,
  <Profile lourdes.leonett (12992011763)>,
  <Profile regioneducativaaltosmir (3982252947)>,
  <Profile modesto_je1 (11125121734)>,
  <Profile espijos (3621870219)>,
  <Profile fonaamazonas_oficial (48869419680)>,
  <Profile aguzmanr90 (2056035841)>,
  <Profile drraulin (565390815)>,
  <Profile anaodalizdelgado (1966293755)>,
  <Profile lugoingridsanchez (53097722648)>,
  <Profile queralesisa (3921668645)>,
  <Profile wendygomez7430 (4545217995)>,
  <Profile marciamiranda3567 (1348342205)>,
  <Profile dr_rommel_martinezh (9363886127)>,
  <Profile dpdmiranda (7160907183)>]}

# Get Diffs
Previous snapshot of posts, comments, likes etc are loaded from disk. 
Current snapshots are gotten from Instagram. Report any deleted or new posts.


In [61]:
def get_posts_diff(target_user):
    # Obtain set of posts that are on hard disk
    offline_posts = set(filter(lambda s: isinstance(s, Post),
                           (load_structure_from_file(L.context, file)
                            for file in (
                                glob(f'{target_user}/*.json.xz') + 
                                glob(f'{target_user}/*.json')))))

    post_iterator = Profile.from_username(L.context, target_user).get_posts()
    online_posts = set(post_iterator)

    new_posts = online_posts - offline_posts
    if online_posts - offline_posts:
        print("Not yet downloaded posts:")
        print(" ".join(str(p) for p in (new_posts)))
    for post in new_posts:
        # 2022-07-22_16-02-32_UTC.json
        timestamp = post.date_utc
        instaloader.save_structure_to_file(post, f'{target_user}/{timestamp}.json)')

    deleted_posts = offline_posts - online_posts
    if deleted_posts:
        print("Deleted posts:")
        print(" ".join(str(p) for p in (deleted_posts)))
                                           
    # __eq__ only comparse shortcodes https://github.com/instaloader/instaloader/blob/3cc29a4ceb3ff4cd04a73dd4b20979b03128f454/instaloader/structures.py#L182
    # Now we need to dive deeper into comments and likes from previously existing posts.
    # Compare the online and offline comments and likes
    online_shortcode_dict = {x.shortcode: x for x in online_posts}
    offline_shortcode_dict = {x.shortcode: x for x in offline_posts}
    intersection_posts = online_posts.intersection(offline_posts)
    new_interactions = {'likes': {}, 'comments': {}}
    deleted_interactions = {'likes': {}, 'comments': {}}
    for post in intersection_posts:
        online_interactions = get_likes_and_comments(
            [online_shortcode_dict[post.shortcode]])
        offline_interactions = get_likes_and_comments(
            [offline_shortcode_dict[post.shortcode]])
        new_interactions['likes'][post.shortcode] = set(online_interactions['likes']) \
            - set(offline_interactions['likes'])
        deleted_interactions['likes'][post.shortcode] = set(offline_interactions['likes']) \
            - set(online_interactions['likes'])

    return new_posts, deleted_posts


## Saime shortcodes

In [96]:
# Manually gathered shortcodes (right click) on homepage
saime_post_shortcodes = [
    'CfPAvzkOPq6',
    'Cev_amPu-3X',
    'CemmG8_MIMb',
    'Ceml3JbMHiT',
    'CeiN4KuuDJf',
    'CeiNK0DuLXr',
    'CeiL_uoupB7',
    'Ceb2mWaD2MD',
    'CdNLaPmussG',
    'CdAVhkssfpB',
    'Cc6u6lUspmS',
    'Cc6h4EYslRe'
]

saime_video_shortcodes = [
    'Ceb2mWaD2MD',
    'CZminxDDJl2',
    'CZi3_4kAdRA',
    'CXjYn8Qp39V',
    'CWd85Z2D1nG',
    'CVT9HaMjWLA',
]


In [97]:
shortcodes = [
'CfPAvzkOPq6',
'Cev_amPu-3X',
'CemmG8_MIMb',
'Ceml3JbMHiT',
'CeiN4KuuDJf',
'CeiNK0DuLXr',
'CeiL_uoupB7',
'Ceb2mWaD2MD',
'CdNLaPmussG',
'CdAVhkssfpB',
'Cc6u6lUspmS',
'Cc6h4EYslRe',
'Ceb2mWaD2MD',
'CZminxDDJl2',
'CZi3_4kAdRA',
'CXjYn8Qp39V',
'CWd85Z2D1nG',
'CVT9HaMjWLA',
'CTJN_sBjavf',
'CTAmn2Pphl5',
'CSyNe_0jNvq',
'CSpq4fLD3HE',
'CSfnrlHDTi2',
'CSZ0DRyj1tB',
]

set(shortcodes).intersection(set(saime_post_shortcodes))

{'Cc6h4EYslRe',
 'Cc6u6lUspmS',
 'CdAVhkssfpB',
 'CdNLaPmussG',
 'Ceb2mWaD2MD',
 'CeiL_uoupB7',
 'CeiN4KuuDJf',
 'CeiNK0DuLXr',
 'Ceml3JbMHiT',
 'CemmG8_MIMb',
 'Cev_amPu-3X',
 'CfPAvzkOPq6'}

In [98]:
set(shortcodes).intersection(set(saime_video_shortcodes))

{'CVT9HaMjWLA',
 'CWd85Z2D1nG',
 'CXjYn8Qp39V',
 'CZi3_4kAdRA',
 'CZminxDDJl2',
 'Ceb2mWaD2MD'}

In [99]:
len(shortcodes)

24

## Post Comments Unit Test 2859783193106312663 
June 18, first picture on square grid profile. 

https://www.instagram.com/p/Cev_amPu-3X/
```
47 mins ago
yuramyborges
Buenas tardes disculpe quería saber sobre de un pasaporte que saqué el 
11 de marzo acá en Ecuador pero no me ha llegado solamente me llegó el 
anuncio de impresión gracias
```
```
3d ago
ccqhud270
Buenas tardes quisiera informacion sobre un pasaporte, 
que estoy necesitando, sería express se encuentra en Chile y quiere 
regresar a Venezuela. Ya su pasaporte tiene dos prórrogas y se vence en 
Marzo 2023 . Agradecida de antemano me puedan orientar
```

## Video Comments Unit Test 2854114918780592899

In [104]:
media_id = 2854114918780592899
get_comments_user_text(media_id)

/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'host.docker.internal'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'user_id': 51942343913,
  'comment_create_at_utc': '2022-06-07 22:36:56',
  'text': 'Los pasaportes de la Urbina, por favor alguien q me conteste, por favor',
  'username': 'josegallardo2909',
  'full_name': 'José gallardo',
  'child_comment_count': 0,
  'child_comments': []},
 {'user_id': 5724296318,
  'comment_create_at_utc': '2022-06-05 20:25:02',
  'text': 'La gente se salió de Venezuela 😂😂',
  'username': 'giseleli1',
  'full_name': 'Gisela Roa',
  'child_comment_count': 0,
  'child_comments': []}]

In [ ]:
### Highlighted Data Captured

Data
Denuncio cuenta IG @saime_redoficial estafador con 
CI 24.237.821 
se llama Luis Ramón Gonzalez Moreno 
cuenta bco vzla con 
celular 04120346974 es funcionario 
Saime que roba a usuarios en oficina plaza Caracas 
y la mamá en Saime sede Propatria


**Facebook**

Saime Pasaporte (Official)  
Joshua Fernando Castro Reyes  
No es por nada eso es una estafa saben el tipo es un venezolano que asta ami persona estafó me dijo que publicará eso mismo y que me pagaría 10 mil pesos por día y como uno anda nesecitado de trabajo ps cuando lo resulte cobrando nunca me pago asta el día de hoy se desentendido así que ya saben no caigan en cuento es una estafa

# References

https://towardsdatascience.com/discover-insights-from-your-instagram-business-account-with-facebook-graph-api-and-python-81d20ee2e751

# Still need to port these functions

## Post Comments

In [165]:
def request_follow(user_id):
    '''
    We need a non-bot account likely of being followed back
    '''
    burp0_url = f"https://www.instagram.com/web/friendships/{user_id}/follow/"
    follow_request = get_burp(burp0_url, method="POST")
    return follow_request
    
def post_comment_by_id(media_id, comment, dry_run=False):

    burp0_url = f"https://www.instagram.com:443/web/comments/{media_id}/add/"
    # TODO manage cookies better
    burp0_data = {"comment_text": comment, "replied_to_comment_id": ''}
    if dry_run:
        print(f'dry-run, would have posted {comment} to media_id {media_id}')
        return {'media_id': 'dry_run', 
                'comment': 'dry_run', 
                'status': 'dry_run'}
    else:
        response = post_session.post(burp0_url, data=burp0_data)
    if response and isinstance(response, requests.Response):
        if (response.status_code == 200):
            return response.json()
        else:
            return response.text
    else:
        print('failed to post_comment by id')
        print(response.text)
        print(response)
        print(type(response))
        return None
    
def post_comment(user, comment, dry_run=False):
    '''
    Post a new comment if we haven't before
    '''
    with open('bot_comments.json', 'r') as f:
        bot_comments = json.load(f)
    media_comment = get_most_recently_commented_on_post(user)
    username = user['username']
    if not media_comment:
        print(f'No media found for {username} to comment on')
        print(request_follow(user['id']))
        return None
    media_id = media_comment['media_id']
    commented_usernames = [x['username'] for x in bot_comments]
    print('commented_usernames', commented_usernames)
    if not media_id:
        print('No media_ids, possibly private. Request follow')
        request_follow(username)
    else:
        shortcode = media_comment['shortcode']
        if username in commented_usernames:
            print(f'''Already posted to media_id {media_id}
            shortcode: {shortcode}, username: {username}''')
            return None
        print(f'Posting to media_comment: {media_comment}')
        response = post_comment_by_id(media_id, comment, dry_run=dry_run)
        if not response:
            print(f'Could not post to {username}. No comments found for {media_id}')
            return None
        if response['status'] == 'ok':
            bot_comments.append({'username': username, 
                                 'media_id': media_id, 
                                 'shortcode': shortcode, 
                                 'comment': comment,
                                 'comment_id': response['id'],
                                 'created_time': response['created_time']
                                })
            print(f'posted comment to {username} at {shortcode}')
            with open('bot_comments.json', 'w') as f:
                json.dump(bot_comments, f)
            return response['id']
        else:
            if not dry_run:
                print('Post comment failed: {response}')
    print(f'Post comment failed for media_id {media_id}')
    return None

def delete_comment(media_id, comment_id):
    burp0_url = f"https://www.instagram.com:443/web/comments/{media_id}/delete/{post_id}/"
    response = session.post(burp0_url, headers=burp0_headers, cookies=burp0_cookies)
    return response

def get_most_recently_commented_on_post(user):
    '''
    To contact a user, we will find there most recent post with
    a comment and 
    '''
    since_date = '00-00-00'
    latest_comments = get_latest_commenters(user, since_date)
    return latest_comments[0] if len(latest_comments) > 0 else None
    

comment = 'denuncia https://enlabajaita.com/2021/08/22/reportan-cuenta-falsa-en-instagram-la-cual-se-esta-haciendo-pasar-por-la-del-saime-oficial/'

#post_id = post_comment('ibangely3544', comment)
#print(post_id)


In [ ]:
cardinals_id = '51240459'
req_follow = request_follow(cardinals_id)
req_follow

In [71]:
post_comment_by_id(test_media_ids[0], comment)

failed to post_comment by id
<Response [403]>


In [ ]:
get_most_recently_commented_on_post(
    get_user_profile('od.jeanvieira')['data']['user'])

In [ ]:
response = delete_comment('2862155623225157033', '17928474998210663')
response

# Warn Users

In [ ]:
def warn_users(target_user, comment, bot_comments, date, max_commenters=10, dry_run=True):
    not_commentable = []
    commented_on = []
    skipped = []
    bot_comments_usernames = [x['username'] for x in bot_comments]
    latest_commenters = get_latest_commenters(target_user, date)
    print(f'Found {len(latest_commenters)} comments')
    if not latest_commenters:
        not_commentable.append((commenter_username, result))
    if max_commenters and max_commenters < len(latest_commenters):
        latest_commenters = latest_commenters[0:max_commenters]
    for commenter in latest_commenters:
        commenter_user = get_user_profile(commenter['username'])
        commenter_user = commenter_user['data']['user']
        commenter_username = commenter_user['username']
        if commenter_username in bot_comments_usernames:
            print('already processed commenter_username ', commenter_username)
            skipped.append(commenter_username)
            continue
        result = post_comment(commenter_user, 
                     comment, 
                     dry_run=dry_run
                    )
        if result and hasattr(result, 'status_code'):
            if result.status_code > 299:
                not_commentable.append((commenter_username, result))
            else:    
                commented_on.append((commenter_username, result))
        elif isinstance(result, dict):
            not_commentable.append((commenter_username, result))
        else:
            not_commentable.append((commenter_username, result))
            print(commenter_username, 
                  'result is not a valid requests response ', 
                  result)

    return commented_on, not_commentable, skipped
    
with open('bot_comments.json', 'r') as f:
    bot_comments = json.load(f)
date = '2022-06-01'
target_user = user
max_commenters = 10
denuncia = f'''hola. Vi tu commentario en la pagina de pasaportes.saime.vzl. 
Te aviso que esta un enstafador, que reciben tu dinero y nunca te dan nada. 
https://www.facebook.com/Denuncia-SAIME-pasaporte-estafas-100816476034483'''
commented_on, not_commentable, skipped = warn_users(
    target_user, denuncia, bot_comments, date, 
    max_commenters=max_commenters, dry_run=False)


## Search

In [135]:
get_user_by_search(username)
target_searches = {}
target_searches['pasaportes'] = search_users("pasaportes")
sorted(target_searches['pasaportes'])

['citas.pasaporte.med',
 'fastvisa.vyp',
 'historiasypasaportes',
 'nikos_pass_',
 'pas_aportes',
 'pasa_portesvzla',
 'pasaportepma',
 'pasaportes',
 'pasaportes.citas.medellin',
 'pasaportes.citascali',
 'pasaportes.saime.vzl',
 'pasaportes.saltillo',
 'pasaportes_.venezolanos',
 'pasaportes_antioquia',
 'pasaportes_bogota',
 'pasaportes_chile_ya',
 'pasaportes_col',
 'pasaportes_colombia',
 'pasaportes_magdalena',
 'pasaportes_medellin',
 'pasaportes_prorroga',
 'pasaportes_tulua',
 'pasaportes_valle',
 'pasaportesadestino',
 'pasaportescolombia',
 'pasaportesefardi',
 'pasaportesindestino',
 'pasaportesmag',
 'pasaportespvat',
 'pasaportesrd',
 'pasaportestudio',
 'pasaportesvenezolano',
 'pasaportesvzlauy',
 'pasaportesyciudadanias',
 'porta.typo',
 'porta_pasaporte',
 'porta_pasaportes',
 'portapasaportes',
 'portutramite',
 'san.tiago_93',
 'sellando.pasaportes',
 'tramitedevisaypasaportes',
 'visasypasaportestueme']

In [136]:
target_searches['pasaportes vz'] = search_users('pasaportes vz')
sorted(target_searches['pasaportes vz'])

['alepasaportes12',
 'pasa_portesvzla',
 'pasaporte_saime.vz',
 'pasaportes.pvat',
 'pasaportes.saime.vzl',
 'pasaportes.vzla',
 'pasaportes.vzla1',
 'pasaportes_valle',
 'pasaportes_vzla',
 'pasaportesccsymas',
 'pasaportesextraviadosvzla',
 'pasaportesvzla',
 'pasaportesvzlauy',
 'pasaportesytramitesvenezuela',
 'passaportesexpresvzla',
 'prorrogapasaportesvzla',
 'saime.pasaporte.vzla',
 'tramitespasaportesvzlaa',
 'tramitesvisas_pasaportes_svaca']

In [156]:
sorted(search_users('saime pasaporte'))

['asesoria_saime_pasaporte',
 'franw_passaport',
 'gestionsaime_',
 'pasaport_vzla',
 'pasaporte_saime.vz',
 'pasaportes.saime.vzl',
 'pasaportesaimevv',
 'saime.pasaporte.vzla',
 'saime.pasaporte2',
 'saime.pasaportess',
 'saime_express',
 'saime_pasaporte.com.ve',
 'saime_pasaporte1',
 'saime_pasaporte21',
 'saime_pasaporte_',
 'saime_pasaporte_ve',
 'saimegestiones',
 'saimepasaporte2019',
 'saimepasaporteweb',
 'spasaporte']

In [ ]:
def get_interacting_users(targets, date):
    interacting_users_by_target = {}
    for target_username in targets:
        print("getting commenters for ", target_username)
        target_user = get_user_profile(target_username)['data']['user']
        interacting_users_by_target[target] = get_latest_commenters(
                target_user, date)
    # TODO Add likes
    
    return interacting_users_by_target

def get_interactions(interacting_users_by_target):
    interactions = {}
    for owner, interaction_user_values in interacting_users_by_target.items():
        interaction_users = [x['username'] for x in interaction_user_values]
        print(owner, interaction_users)
        for target in interacting_users_by_target.keys():
            if target in interaction_users:
                interactions[owner].append(target)
    return interactions
                
interacting_users = get_interacting_users(
    target_searches['pasaportes vz'][0:3], '2022-05-01')
print(interacting_users)
get_interactions(interacting_users)